In [1]:
import control as ctrl
import numpy as np

# Análisis de los sistemas en su forma de espacio de estados

Supongamos que tenemos la siguiente función transferencia:

In [2]:
G=ctrl.tf([1,1],[1,2,3])
G

TransferFunction(array([1, 1]), array([1, 2, 3]))

De los apuntes de teoría podemos escribir una representación de espacios de estados en la forma canónica de controlabilidad que tendrá el mismo comportamiento externo de $G$ de la siguiente manera :

In [3]:
Ac=np.matrix([[-2, -3],[1,0]])
Ac

matrix([[-2, -3],
        [ 1,  0]])

Antes de seguir, verificamos que los polos de $G$ sean los autovalores de $\mathbf {Ac}$

In [4]:
print(G.pole())
print(np.linalg.eigvals(Ac))

[-1.+1.41421356j -1.-1.41421356j]
[-1.+1.41421356j -1.-1.41421356j]


Ahora definimos el resto de las matrices

In [5]:
Bc=np.matrix("1;0")
Bc

matrix([[1],
        [0]])

In [6]:
Cc=np.matrix("1 1")
Cc

matrix([[1, 1]])

In [7]:
Dc=0

Vamos ahora a definir un sistema $sys$ que se comporte igual que $G$

In [8]:
sys_c=ctrl.ss(Ac,Bc,Cc,Dc)

Si $sys$ tiene los mismos polos, los mismos ceros y la misma ganancia en estado estacionario, entonces se comportará igual que $G$.

Veamos los polos:

In [9]:
sys_c.pole()

array([-1.+1.41421356j, -1.-1.41421356j])

Ahora los ceros:

In [10]:
sys_c.zero()

array([-1.+0.j])

Y la ganancia en estado estacionario en particular (o en cualquier frecuencia en general):

In [11]:
sys_c.dcgain()

0.3333333333333333

In [12]:
G.dcgain()

0.3333333333333333

Por lo tanto podemos ver que la función transferencia de $sys$ será la misma que la de $G$. Es decir, $sys$ se comporta **externamente** igual que $G$. Verificamos

In [13]:
ctrl.tf(sys_c)

TransferFunction(array([1., 1.]), array([1., 2., 3.]))

Vamos ahora a definir el sistema en espacio de estados, que se comporte como $G$ en la forma canónica de observabilidad. De la teoría sabemos que:

In [14]:
Ao=Ac.T
Bo=Cc.T
Co=Bc.T
Do=Dc

In [15]:
sys_o=ctrl.ss(Ao,Bo,Co,Do)

In [16]:
sys_o.pole()

array([-1.+1.41421356j, -1.-1.41421356j])

In [17]:
sys_o.zero()

array([-1.+0.j])

In [18]:
sys_o.dcgain()

0.3333333333333333

In [19]:
ctrl.tf(sys_o)

TransferFunction(array([1., 1.]), array([1., 2., 3.]))

In [20]:
ctrl.tf(sys_o)

TransferFunction(array([1., 1.]), array([1., 2., 3.]))

El módulo de control de Python puede transformar una función transferencia a una forma de estados (no necesariamente en alguna forma canónica). Esto se hace:

In [21]:
sys=ctrl.ss(G)
sys

StateSpace(array([[-2.00000000e+00,  3.00000000e+00],
       [-1.00000000e+00,  3.33066907e-16]]), array([[1.],
       [0.]]), array([[ 1., -1.]]), array([[0.]]))

Además Python puede transformar un sistema en espacio de estados a alguna de las formas canónicas.

Por ejemplo para escribir $sys$ (que está ahora en espacio de estados) en su forma canónica de controlabilidad podemos hacer

In [22]:
sys_c, Tc=ctrl.canonical_form(sys, 'reachable') # controlable
sys_c, Tc

(StateSpace(array([[-2., -3.],
        [ 1.,  0.]]), array([[1.],
        [0.]]), array([[1., 1.]]), array([[0.]])),
 array([[ 1.00000000e+00, -2.22044605e-16],
        [-0.00000000e+00, -1.00000000e+00]]))

Notar que esta función necesita como primer argumento un sistema de espacios de estados.

Si queremos la forma canónica de observabilidad podemos hacer:

In [23]:
sys_o,To = ctrl.canonical_form(sys, 'observable')
sys_o, To

(StateSpace(array([[-2.,  1.],
        [-3.,  0.]]), array([[1.],
        [1.]]), array([[1., 0.]]), array([[0.]])),
 array([[ 1., -1.],
        [ 1.,  1.]]))

Existe otra forma canónica de representación en espacio de estados que se la conoce como forma canónica modal. Esta la podemos obtener haciendo:

In [24]:
sys_m, Tm = ctrl.canonical_form(sys, 'modal')
sys_m, Tm

(StateSpace(array([[-1.        ,  3.41421356],
        [-0.58578644, -1.        ]]), array([[ 0.92387953],
        [-0.38268343]]), array([[ 0.5411961 , -1.30656296]]), array([[0.]])),
 array([[ 0.92387953, -0.38268343],
        [ 0.38268343,  0.92387953]]))

Esta forma canónica lo que hace es aislar los modos unos de otros. Es decir cada variables de estado aparece en la diagonal. Sin embargo, como vemos en este caso, no tenemos una forma diagonal de la matriz $\mathbf A$. Esto sucede por que hay casos que no se puede o no se desea aislarlos:
- en caso de que se tengan autovalores complejos conjugados, no se aíslan para lograr tener una matriz $\mathbf A$, $\mathbf B$ y $\mathbf C$ a coeficiente reales.
- en caso de que sean multiples no se pueden aislar por que tienen el mismo autovector y no se pueden usar estos para la transformación. Para estos casos se utiliza la forma de Jordan.

Vamos a ver otro ejemplo:

In [25]:
s=ctrl.tf('s')
G2 = 1/((s+1)*(s+2)*(s+3))
G2

TransferFunction(array([1]), array([ 1,  6, 11,  6]))

Vemos que esta función transferencia tiene los polos en -1, -2 y -3. Transformemos a espacio de estados usando la función `ss`.

In [26]:
sys2 = ctrl.ss(G2)
sys2

StateSpace(array([[ -6. ,   1.1,  -0.6],
       [-10. ,   0. ,   0. ],
       [  0. ,  -1. ,   0. ]]), array([[-1.],
       [ 0.],
       [ 0.]]), array([[ 0. ,  0. , -0.1]]), array([[0.]]))

Veamos al sistema en su forma canónica de controlabilidad.

In [27]:
ctrl.canonical_form(sys2, 'reachable')

(StateSpace(array([[ -6., -11.,  -6.],
        [  1.,   0.,   0.],
        [  0.,   1.,   0.]]), array([[1.],
        [0.],
        [0.]]), array([[-6.25762068e-18, -6.30808537e-17,  1.00000000e+00]]), array([[0.]])),
 array([[-1.00000000e+00,  6.86319688e-17,  2.44249065e-16],
        [ 3.55271368e-17,  1.00000000e-01,  9.43689571e-18],
        [-6.25762068e-18,  6.30808537e-18, -1.00000000e-01]]))

Ahora lo podemos ver en la forma canónica de observabilidad

In [28]:
ctrl.canonical_form(sys2, 'observable')

(StateSpace(array([[ -6.,   1.,   0.],
        [-11.,   0.,   1.],
        [ -6.,   0.,   0.]]), array([[0.],
        [0.],
        [1.]]), array([[1., 0., 0.]]), array([[0.]])),
 array([[ 0.0000000e+00, -4.4408921e-18, -1.0000000e-01],
        [-0.0000000e+00,  1.0000000e-01, -6.0000000e-01],
        [-1.0000000e+00,  6.0000000e-01, -1.1000000e+00]]))

Finalmente obtengamos la forma canónica modal:

In [29]:
sys2_m, Tm =ctrl.canonical_form(sys2, 'modal')
sys2_m.A


array([[-3.00000000e+00, -1.35377372e-14,  2.02533447e-15],
       [ 2.83137424e-14, -2.00000000e+00, -5.23993893e-15],
       [ 8.83000508e-15,  5.63654650e-15, -1.00000000e+00]])

```{note}
- La función `canonical_form` devuelve un sistema en la forma canónica solicitada y la matriz de transformación necesaria para obtenerlo
- en la diagonal de la matriz  $\mathbf A$ de la forma canónica modal tenemos los modos del sistema (polos de $G$ o autovalores de $A$)
- fuera de la diagonal $\mathbf A$ tenemos ceros o valores muy cercanos a ceros (debido a problemas numéricos en el cálculo).
- En caso de tener autovalores repetidos, tendremos una matriz $\mathbf{A}$ diagonal por bloques. Los valores fuera de la diagonal serán 1.
- En general, los autovalores complejos conjugados tampoco se ponen en la diagonal, para evitar tener tener matrices con valores complejos.
```

Verificamos el resto de las matrices:

In [30]:
sys2_m.B

array([[ 16.43928222],
       [-22.71563338],
       [ -7.08872344]])

In [31]:
sys2_m.C

array([[ 0.03041495,  0.04402255, -0.07053456]])

In [32]:
sys2_m.D

array([[0.]])

En esta forma canónica, la matriz $B$ como la conexión de la entrada con el modo del sistema . Y de forma análoga a la matriz $C$ como la conexión del modo con la salida del sistema.

## Ejercicio análisis de Controlabilidad y Observabilidad

Sunpongamos que se tienen las sieguentes funciones transferencias $G_1$ y $G_2$:

## Ejemplo de conexión de sistemas

In [33]:
G1=ctrl.tf(1,[1,1])
G2=ctrl.tf([1,1],[1,2])
G1

TransferFunction(array([1]), array([1, 1]))

In [34]:
G2

TransferFunction(array([1, 1]), array([1, 2]))

Podemos ver que G1 tiene un polo en -1 y G2 tiene un polo en -2 y un cero en -1.

Ahora supongamos que queremos un nuevo sistema que:
- conecta la salida de $G1$ con la entrada de $G2$
- la salida es la salida de $G2$
- la entrada es la entrada de $G1$

Para eso vamos a usar una función que se llama `connect`. Esta utiliza sistemas en espacio de estados, entonces hacemos:

In [35]:
sys1 = ctrl.ss(G1)
sys2= ctrl.ss(G2)

Y luego realizamos las conexiones:

In [36]:
sys_c = ctrl.connect(ctrl.append(sys1, sys2),[[2,1]],[1],[2])
sys_c

StateSpace(array([[-1.,  0.],
       [ 1., -2.]]), array([[1.],
       [0.]]), array([[ 1., -1.]]), array([[0.]]))

Analizamos  los polos y ceros del sistema ya interconectado

In [37]:
sys_c.pole()

array([-2., -1.])

In [38]:
sys_c.zero()

array([-1.+0.j])

Vemos que el sistema conserva los mismos polos y los mismos ceros que el original, lo cual era de esperarse, ya que su función transferencia sería $G1.G2$

Obtengamos la forma canónica de controlabilidad del sistema

In [39]:
sys_c, Tc=ctrl.canonical_form(sys_c, 'reachable')
sys_c

StateSpace(array([[-3., -2.],
       [ 1.,  0.]]), array([[1.],
       [0.]]), array([[1., 1.]]), array([[0.]]))

Podemos ver que las matrices $ \mathbf A$, $\mathbf B$, $\mathbf C$ y $\mathbf D$ coinciden con lo visto en teoría.

Ahora obtengamos la forma canónica de observabilidad:

In [40]:
ctrl.canonical_form(sys_c, 'observable')

ValueError: Transformation matrix singular to working precision.

Podemos ver que este sistema no puede ser escrito en su forma canónica de observabilidad. Esto sucede por que el sistema no es observable

```{admonition} Interpretación
:class: hint

Un sistema observable necesita tener evidencia de lo que sucede con cada modo en al menos una salida del sistema. Como el modo en -1 se ve completamente tapado por el cero en esa misma frecuencia este sistema no es observable por que no tiene ninguna evidencia del modo en -1 a la salida del mismo.

```

Ahora hagamos la conexión al revés (primero $G1$ y luego $G2$):

In [41]:
sys_o = ctrl.connect(ctrl.append(sys1, sys2),[[1,2]],[2],[1])
sys_o

StateSpace(array([[-1., -1.],
       [ 0., -2.]]), array([[1.],
       [1.]]), array([[1., 0.]]), array([[0.]]))

Obtengamos la forma canónica observable:

In [42]:
sys_o, To=ctrl.canonical_form(sys_o, 'observable')
sys_o

StateSpace(array([[-3.,  1.],
       [-2.,  0.]]), array([[1.],
       [1.]]), array([[1., 0.]]), array([[0.]]))

Y ahora la forma canónica controlable:

In [43]:
ctrl.canonical_form(sys_o, 'reachable')

ValueError: System not controllable to working precision.

Podemos ver que ahora no podemos obtener la forma canónica de observabilidad.

```{admonition} Interpretación
:class: hint

La razón de esto es que no se puede excitar el modo -1, que ahora quedó tapado desde el lado de la entrada. Toda excitación que tenga "frecuencia generalizada -1" será tapada por el cero en -1. Por lo tanto este sistema no es controlable, ya que no puede excitar desde esa entrada el modo -1.
```

Es importante destacar que ambos sistemas presentan la misma función transferencia $G1.G2$, pero uno es solo controlable y el otro es solo observable. Esto se debe a que la controlabilidad y la observabilidad son propiedades internas de un sistema. 



```{attention}
La **controlabilidad** y la **observabilidad**  no pueden ser decididas a partir de modelos externos de un sistema como son las funciones transferencias.
```

Estas dos mismas propiedades pueden ser analizadas con la forma modal. Tomemos el sistema $sys_C$ donde la entrada esta conectada a $G1$ y la salida a $G2$:

In [44]:
sys_mc, Tmc = ctrl.canonical_form(sys_c, 'modal')
sys_mc

StateSpace(array([[-2.00000000e+00, -1.79022500e-16],
       [ 4.52542462e-17, -1.00000000e+00]]), array([[-2.23606798],
       [-1.41421356]]), array([[-0.4472136,  0.       ]]), array([[0.]]))

Podemos ver que la matriz $\mathbf B$ tiene todos los valores distintos de 0. Esto significa que todos los modos están conectados a la entrada, por lo que deducimos que el sistema es controlable.

Pero podemos ver que $\mathbf C$ tiene un 0 en el primer elemento. Y que en la diagonal de $ \mathbf A$ el primer elemento es -1. Esto quiere decir que no hay ninguna evidencia del modo -1 en la salida por que el sistema no es observable.

Probemos ahora con la segunda conexión:

In [45]:
sys_mo, Tmo = ctrl.canonical_form(sys_o, 'modal')
sys_mo

StateSpace(array([[-2.00000000e+00,  9.05084924e-17],
       [-8.95112501e-17, -1.00000000e+00]]), array([[-1.41421356],
       [-0.        ]]), array([[-0.70710678, -0.4472136 ]]), array([[0.]]))

Podemos ver ahora que la matriz $\mathbf C$ no tiene ningún valor igual a 0, por lo que ahora el sistema sería observable (todos los modos están presentes de alguna manera en la salida). Pero tenemos un 0 en la primer fila de $\mathbf B$, que se corresponde con el modo en -1. Esto quiere decir que no podemos excitar este modo. Por lo tanto no es controlable.

```{important}
La controlabilidad y la observabilidad pueden ser evaluadas con el sistema en su forma **canónica modal** analizando las relaciones  de la matrices $\mathbf{B}$ y $\mathbf{C}$ con la matriz $\mathbf{A}$.
```